In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import recall_score, classification_report
from sklearn.impute import SimpleImputer, KNNImputer
from scipy.stats import boxcox
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVC, SVR
from precision_recall_cutoff import precision_recall_cutoff

s3 = boto3.resource('s3')
bucket_name = 'grant-gonnerman-data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'turnover.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading data file
turnover = pd.read_csv(file_content_stream)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [2]:
# changing sales and salary dummies
turnover = pd.concat([turnover.drop(columns = ['sales', 'salary'], axis =  1 ), pd.get_dummies(turnover[['sales', 'salary']])], axis = 1)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [3]:
# engineering interactions
turnover['interaction_1'] = np.where((turnover['satisfaction_level'] >= .115) & (turnover['satisfaction_level'] <= .465) & (turnover['number_project'] > 2.5), 1, 0)
turnover['interaction_2'] = np.where((turnover['satisfaction_level'] <= .465) & (turnover['number_project'] <= 2.5) & (turnover['last_evaluation'] <= .575), 1, 0)
turnover['interaction_3'] = np.where((turnover['satisfaction_level'] > .465) & (turnover['time_spend_company'] <= 4.5) & (turnover['average_montly_hours'] <= 290.5), 1, 0)

In [4]:
# defining imput and target
x = turnover[['interaction_3', 'interaction_1', 'satisfaction_level', 'time_spend_company', 'number_project']]
y = turnover['left']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [5]:
## Defining scaler for logistic regression
scaler = MinMaxScaler()
## Scaling the data
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x.columns)

In [6]:
# grid search cv
RF_param_grid = {'n_estimators': [100, 300, 500],
                 'min_samples_split': [10, 15],
                 'min_samples_leaf': [5, 7],
                 'max_depth': [3, 5, 7]}

# running grid search 
rf_grid = GridSearchCV(estimator = RandomForestClassifier(), param_grid = RF_param_grid, cv = 5, scoring = 'f1').fit(x_train, y_train)

# extracting best hyper params
rf_grid.best_params_

{'max_depth': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 500}

In [7]:
# building random forest
rf_md = RandomForestClassifier(n_estimators = 100, max_depth = 7, min_samples_leaf = 5, min_samples_split= 15).fit(x_train, y_train)

# predicting on test
rf_pred = rf_md.predict_proba(x_test)[:,1]

# likilyhookd to label
rf_labels = precision_recall_cutoff(y_test, rf_pred)

print(classification_report(y_test, rf_labels))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2286
           1       0.96      0.92      0.94       714

    accuracy                           0.97      3000
   macro avg       0.97      0.95      0.96      3000
weighted avg       0.97      0.97      0.97      3000



In [8]:
# grid search cv
SVM_param_grid = {'kernel': ['rbf', 'poly', 'sigmoid'],
                    'C': [0.01, 0.1, 1, 10],
                    'gamma': [0.001, 0.01, 0.1, 1]}

# running grid search 
svc_grid = GridSearchCV(estimator = SVC(probability = True), param_grid = SVM_param_grid, cv = 5, scoring = 'f1').fit(x_train, y_train)

# extracting best hyper params
svc_grid.best_params_

{'C': 10, 'gamma': 1, 'kernel': 'rbf'}

In [10]:
# building random forest
svc_md = SVC(kernel = 'rbf', C = 10, gamma = 1, probability = True).fit(x_train, y_train)

# predicting on test
svc_pred = svc_md.predict_proba(x_test)[:,1]

# likilyhookd to label
svc_labels = precision_recall_cutoff(y_test, svc_pred)

print(classification_report(y_test, svc_labels))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98      2286
           1       0.92      0.92      0.92       714

    accuracy                           0.96      3000
   macro avg       0.95      0.95      0.95      3000
weighted avg       0.96      0.96      0.96      3000



In [ ]:
# based on the classification reports the performace of the models are very similar. but I would choose random forest because it looks to perform a tad better